In [ ]:
########new dataset
#images
import argparse
import pickle
from torchvision.models import detection
import torch
import cv2
import numpy as np
import matplotlib.pyplot as plt 
from PIL import Image
from stacked_hourglass.datasets import generic
import argparse
import glob
from albumentations.core.composition import KeypointParams
from torchvision import datasets
from torch.utils.data import DataLoader
from torchvision import transforms
import torch
import numpy as np
from stacked_hourglass.predictor import HumanPosePredictor, GenericPosePredictor
from stacked_hourglass.model import hg2
import cv2
import matplotlib.pyplot as plt
import torch.nn as nn
import csv
import glob
import random
import copy
from PIL import Image
import albumentations as A
from stacked_hourglass.utils import finetune
from stacked_hourglass import train as t
from stacked_hourglass.datasets.mpii import Mpii
from torch.nn import DataParallel
from stacked_hourglass.utils.logger import Logger
import os
from stacked_hourglass.utils.misc import save_checkpoint, adjust_learning_rate
from stacked_hourglass.train import do_training_epoch, do_validation_epoch
import tqdm
import argparse
from stacked_hourglass.datasets.mpii import MPII_JOINT_NAMES
import cv2
from torch.optim.rmsprop import RMSprop
from torch.utils.tensorboard import SummaryWriter
import random

#####################################################################################################
parser = argparse.ArgumentParser(description='Train a stacked hourglass model.')
# Model structure
parser.add_argument('--arch', '-a', metavar='ARCH', default='hg2',
                    choices=['hg1', 'hg2', 'hg8'],
                    help='model architecture')
# Training strategy
parser.add_argument('--input_shape', default=(256, 256), type=int, nargs='+',
                    help='Input shape of the model. Given as: (H, W)')
parser.add_argument('-j', '--workers', default=4, type=int, metavar='N',
                    help='number of data loading workers (default: 4)')
parser.add_argument('--epochs', default=20, type=int, metavar='N',
                    help='number of total epochs to run')
parser.add_argument('--start-epoch', default=0, type=int, metavar='N',
                    help='manual epoch number (useful on restarts)')
parser.add_argument('--train-batch', default=16, type=int, metavar='N',
                    help='train batchsize')
parser.add_argument('--test-batch', default=16, type=int, metavar='N',
                    help='test batchsize')
parser.add_argument('--lr', '--learning-rate', default=2.5e-4, type=float,
                    metavar='LR', help='initial learning rate')
parser.add_argument('--momentum', default=0.9, type=float, metavar='M',
                    help='momentum')
parser.add_argument('--weight-decay', '--wd', default=0, type=float,
                    metavar='W', help='weight decay (default: 0)')
parser.add_argument('--schedule', type=int, nargs='+', default=[200, 201],
                    help='Decrease learning rate at these epochs.')
parser.add_argument('--gamma', type=float, default=0.1,
                    help='LR is multiplied by gamma on schedule.')
# Miscs
parser.add_argument('-c', '--checkpoint', default='checkpoint/', type=str, metavar='PATH',
                    help='path to save checkpoint (default: checkpoint)')
parser.add_argument('--snapshot', default=0, type=int,
                    help='save models for every #snapshot epochs (default: 0)')
parser.add_argument('--resume', default='', type=str, metavar='PATH',
                    help='path to latest checkpoint (default: none)')

#train and eval
lr = args.lr
dic={'train_loss':[],'val_loss':[],'train_acc':[],'val_acc':[],'lr':[],'epoch':[]}
for epoch in range(args.start_epoch,args.epochs):
    lr = adjust_learning_rate(optimiser, epoch, lr, args.schedule, args.gamma)
    # train for one epoch
    batch_size=args.train_batch

    
    train_loss, train_acc = do_training_epoch(train_loader, model, device,optimiser)

    # evaluate on validation set
    valid_loss, valid_acc= do_validation_epoch(val_loader, model, device)
    print('epoch %2d :        Train Loss: %.9f | Valid Loss: %.9f | Train Accuracy: %.3f |  Valid Accuracy: %.3f'%(epoch,train_loss,valid_loss,train_acc,valid_acc))
    
    ############manual show
    dic['train_loss'].append(train_loss*1000)
    dic['val_loss'].append(valid_loss*1000)
    dic['train_acc'].append(train_acc)
    dic['val_acc'].append(valid_acc)
    dic['lr'].append(lr)
    dic['epoch'].append(epoch)

#####################################################################################################
#draw the result with tensorboard
epoch=np.linspace(1,len(dic['lr']),len(dic['lr']))
n=0
plt.subplot(5,1,1)
plt.plot(epoch[n:],dic['train_loss'][n:])
plt.ylabel('train loss')

plt.subplot(5,1,2)
plt.plot(epoch[n:],dic['val_loss'][n:])
plt.ylabel('val loss')

plt.subplot(5,1,4)
plt.plot(epoch[n:],dic['train_acc'][n:])
plt.ylabel('train acc')

plt.subplot(5,1,5)
plt.plot(epoch[n:],dic['val_acc'][n:])
plt.ylabel('val acc')
plt.xlabel('epoch')    